<a href="https://colab.research.google.com/github/lightdarkmaster/Album/blob/main/my-YOLO8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.4/881.4 kB 13.5 MB/s eta 0:00:00


In [2]:
## importing required libraries
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [4]:
## connecting to the google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="vhuhQ5id2ui3NSMKgzPl")
project = rf.workspace("capstone-project-1").project("dataset2-9f0jj")
version = project.version(2)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Dependency ultralytics==8.0.196 is required but found version=8.3.3, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to dataset2-2 in yolov8:: 100%|██████████| 244/244 [00:00<00:00, 6127.32it/s]


In [6]:
train_path_img = "/content/dataset2-2/train/images"
train_path_label = "/content/dataset2-2/train/labels"
val_path_img = "/content/dataset2-2/valid/images"
val_path_label = "/content/dataset2-2/valid/labels"
test_path = "/content/dataset2-2/test"

In [11]:

'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
              continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/dataset2-2/train/images') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 105 images---


  0%|          | 0/84 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '/content/dataset2-2/train/images6_png.rf.37f10684dff7a627ab0bf6e89dd0424c.jpg'

In [12]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.3 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 36.6/107.7 GB disk)


In [13]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/dataset2-2/data.yaml epochs=10 imgsz=640 batch=8 project=/content/drive/MyDrive/yolov8/training_results name=football

Ultralytics 8.3.3 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/dataset2-2/data.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8/training_results, name=football, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, s

In [14]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/football/weights/best.pt conf=0.55 source=/content/dataset2-2/test/images

Ultralytics 8.3.3 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 186 layers, 9,828,825 parameters, 0 gradients, 23.3 GFLOPs

image 1/3 /content/dataset2-2/test/images/16_png.rf.ce9ebdb0f8690acde35db6db5b94340c.jpg: 640x640 12 Less viables, 718.8ms
image 2/3 /content/dataset2-2/test/images/Non-viable00601_png.rf.49f0fd99b96a71640167f2e40188ff96.jpg: 640x640 (no detections), 565.6ms
image 3/3 /content/dataset2-2/test/images/viable00251_png.rf.b6b2ffd01feb16671739d2479f7fb566.jpg: 640x640 (no detections), 603.3ms
Speed: 6.9ms preprocess, 629.2ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [15]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov8/output

In [19]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/football/weights/best.pt conf=0.77 source=/content/drive/MyDrive/LessViable3.MP4

Ultralytics 8.3.3 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 186 layers, 9,828,825 parameters, 0 gradients, 23.3 GFLOPs

video 1/1 (frame 1/535) /content/drive/MyDrive/LessViable3.MP4: 384x640 (no detections), 414.4ms
video 1/1 (frame 2/535) /content/drive/MyDrive/LessViable3.MP4: 384x640 (no detections), 445.0ms
video 1/1 (frame 3/535) /content/drive/MyDrive/LessViable3.MP4: 384x640 (no detections), 407.0ms
video 1/1 (frame 4/535) /content/drive/MyDrive/LessViable3.MP4: 384x640 1 Less viable, 387.8ms
video 1/1 (frame 5/535) /content/drive/MyDrive/LessViable3.MP4: 384x640 1 Less viable, 414.4ms
video 1/1 (frame 6/535) /content/drive/MyDrive/LessViable3.MP4: 384x640 1 Less viable, 440.7ms
video 1/1 (frame 7/535) /content/drive/MyDrive/LessViable3.MP4: 384x640 (no detections), 525.9ms
video 1/1 (frame 8/535) /content/drive/MyDrive/LessViable3.MP4: 384x640 (no detections), 582.2ms
video 1/1 (frame 9/535) /content/drive/MyDrive/LessViable3.MP4: 384x6